<a href="https://colab.research.google.com/github/SanyaMar/-/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import csv
from typing import Any, Tuple, List

import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class MyDataset(Dataset):
    def __init__(self, path_to_annot_file: str, transform: Any=None, train: bool=False, test: bool=False, valid: bool=False) -> None:
        self.path_to_annot_file = path_to_annot_file
        self.transform = transform

        data = pd.read_csv(path_to_annot_file, delimiter=',', usecols=(0, 2), names=('abs_path', 'name'))
        data['mark'] = 0
        data.loc[data['name'] == 'dog', 'mark'] = 1

        X_train_val, X_test, y_train_val, y_test = train_test_split(data['abs_path'], data['mark'], test_size=0.1, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.11, random_state=42)

        if train:
            self.data = pd.DataFrame({'abs_path': X_train, 'mark': y_train})
        elif test:
            self.data = pd.DataFrame({'abs_path': X_test, 'mark': y_test})
        elif valid:
            self.data = pd.DataFrame({'abs_path': X_val, 'mark': y_val})

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        path_to_image = self.data.iloc[index, 0]
        image = cv2.cvtColor(cv2.imread(path_to_image), cv2.COLOR_BGR2RGB)
        label = self.data.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Resize((224, 224)),
                                                    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                                    torchvision.transforms.RandomHorizontalFlip(p=0.3)])

In [ ]:
train_data = MyDataset(path_to_annot_file='/content/drive/MyDrive/CN/paths_1.csv', transform=transforms, train = True)
test_data = MyDataset(path_to_annot_file='/content/drive/MyDrive/CN/paths_1.csv', transform=transforms, test = True)
valid_data = MyDataset(path_to_annot_file='/content/drive/MyDrive/CN/paths_1.csv', transform=transforms, valid = True)

In [ ]:
len(train_data), len(test_data), len(valid_data)

(1602, 201, 198)

In [ ]:
train_data.data['mark'].value_counts()

1    802
0    800
Name: mark, dtype: int64